## Public Demo of IncQuery Server Jupyter Client Extensions

### Preliminaries

#### Setup IQS Connection

Execute the following piece of code to connect to the public IncQuery Server demo instance.

If you have non-guest privileges, you may specify your credentials here.

In [ ]:
import iqs_jupyter
iqs = iqs_jupyter.connect(
    address='https://openmbee.incquery.io/api',
    user='openmbeeguest',
    password='guest'
)

#### Select MMS commit to consider

Run the next code block to display the commit selector widget, and use it to browse around the MMS repository. When you've had your fun, make sure to leave it in a state where a commit is selected from the _IQS4MMS Demos_ org, as we have made sure to pre-index and load those commits in the IQS.

In [ ]:
commit_selector = iqs.jupyter_tools.mms_commit_selector_widget()

The following piece of code assigns the Python name `model` to the MMS commit selected above, and checks whether the model is indeed indexed and loaded by IQS, which is required for the rest of the demo 

In [ ]:
model = commit_selector.value().to_model_compartment()
if model.is_loaded_by_server(iqs):
    print("We may proceed.")
else: 
    print("Model is not indexed&loaded by IQS, so the next demo steps will not work.")
    print(" (Unfortunately, guest users are not allowed to control model indexing.)")
    print("Please select another model from the 'IQS4MMS Demos' org.")

### Model validation

#### Perform validation checks with custom validation rules (progress with indexed models only)

Request and display a validation report for the selected model

In [ ]:
validation_report = iqs.validation.validate_model_compartment(model)
validation_report

#### Visualize custom validation reports via Pandas dataframes and Plot.ly

The following block renders results in Pandas dataframe format, ready for complex client-side manipulation:

In [ ]:
validation_report_df = validation_report.to_data_frame()

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [ ]:
import cufflinks as cf
cf.go_offline()

Preprocess results using Pandas and visualize the output using Plot.ly; this is handy for generating many kinds of custom validation reports.

First, here is a diagram showing the number of violations found for each validation rule:

In [ ]:
validation_report_df.groupby(by=['constraint_element_name','severity']).size().unstack(level=-1).iplot( 
    kind='bar', barmode='stack', colors = iqs_jupyter.validation_color_scale,
    filename='cufflinks/categorical-bar-chart/frequent_diagnostic_rules',
    yTitle='Number of Diagnostics per Diagnostic Rule', title='Diagnostic Report',
)

Next, we do some precomputation with Pandas and plot a chart of violating elements against their total number of violations; if there are multiple severities (e.g. warnings and errors), the bars will be subdivided (stacked bar chart).

In [ ]:
validation_report_elementwise = validation_report_df.groupby(by=['matching_element_relative_id','severity']).size().unstack(level=-1, fill_value=0)
validation_report_elementwise['total'] = validation_report_elementwise.apply(sum, axis=1)
cols_sorted = ['total'] + [severity for severity in iqs_jupyter.validation_diagnostics_ranking if severity in validation_report_elementwise.columns.to_list()]
validation_report_elementwise = validation_report_elementwise.sort_values(by=cols_sorted, ascending=False).drop('total', axis=1)

In [ ]:
validation_report_elementwise.iplot( 
    kind='bar', barmode='stack', colors = iqs_jupyter.validation_color_scale,
    filename='cufflinks/stacked-bar-chart/diagnostics_per_element',
    yTitle='Number of Diagnostics per Violating Element', title='Detailed Diagnostic Report',
)

### Coverage report using custom model queries

#### Define and register custom ad-hoc queries

The custom model queries defined below will discover multiple forms of traceability to _Requirement_ elements, determine the coverage of _Block_ elements by such traceability links, and aggregate coverage metrics for _Package_s containing these _Block_s.

In [ ]:
coverage_query_package = "iqs4mms.demo.coverage"
coverage_query_main = "iqs4mms.demo.coverage.packageCoverage"
coverage_query_code = '''

// SECTION 1: main query

/* 
 * Associates a UML package with 
 *  - the total number of transitively contained SysML Blocks, and 
 *  - the number of strongly and weakly covered blocks among them
 */
pattern packageCoverage(
    pack: Package, 
    totalBlocks: java Integer, 
    stronglyCovered: java Integer, 
    weaklyCovered: java Integer
) {
    totalBlocks     == count find blockInPackage(_, pack);
    stronglyCovered == count find stronglyCoveredBlockInPackage(_, pack);
    weaklyCovered   == count find weaklyCoveredBlockInPackage(_, pack);
}


// SECTION 2: custom in-house definitions for strong and weak coverage

/* 
 * Identifies elements that are strongly covered by a requirement.
 * This definition may be customized according to in-house concept of strong coverage.
 */
pattern stronglyCovered(element: NamedElement) {
    find util.sysml.sysmlRequirement_SatisfiedBy(_, element);
} or {
    find util.sysml.sysmlRequirement_VerifiedBy(_, element);
} or {
    find util.sysml.sysmlRequirement_TracedTo(_, element);
}
/* 
 * Identifies elements that are NOT strongly covered, 
 * but are reachable from a strongly covered element 
 * using one or more custom propagation steps
 */
pattern weaklyCovered(element: NamedElement) {
    find stronglyCovered(otherElement);
    find coveragePropagates+(otherElement, element);
    neg find stronglyCovered(element);
}
/* 
 * Defines custom coverage propagation steps, where 
 * coverage of a 'from' element automatically implies weak coverage of a 'to' element.
 * This definition may be customized according to in-house propagation rules.
 */
pattern coveragePropagates(from: NamedElement, to: NamedElement) {
    // from container to contained part
    Property.owner(part, from);
    Property.aggregation(part, ::composite);
    TypedElement.type(part, to);
} or { 
    // from general superclassifier / block to specific block
    Generalization.general(gen, from);
    Generalization.specific(gen, to);
}

// SECTION 3: helper queries to assemble a report on (transitive) package contents

/* 
 * Associates SysML Blocks with UML packages directly or indirectly containing them.
 */
pattern blockInPackage(block: Class, pack: Package) {
    find util.sysml.sysmlBlock(block, _);    
    Element.owner+(block, pack);
}
/* 
 * Associates strongly covered Blocks with UML packages directly or indirectly containing them.
 */
pattern stronglyCoveredBlockInPackage(block: Class, pack: Package) {
    find stronglyCovered(block);    
    find blockInPackage(block, pack);
}
/* 
 * Associates weakly covered Blocks with UML packages directly or indirectly containing them.
 */
pattern weaklyCoveredBlockInPackage(block: Class, pack: Package) {
    find weaklyCovered(block);    
    find blockInPackage(block, pack);
}


'''

Queries need to be registered on the server before they can be evaluated. _Note: you need elevated privileges to be able to register new queries; for the guest user in this demo, the next part will only work if the query has already been registered; otherwise you will receive a '403 Forbidden' error._

In [ ]:
if coverage_query_main not in iqs.queries.list_queries().query_fq_ns: # skip if already registered
    try: 
        iqs.queries.register_queries_plain_text(coverage_query_code, query_package=coverage_query_package)
    except: 
        print("Query registration not available as guest user; please try again in a few minutes")
else:
    print("Query is already registered; proceed")

To verify, see which queries are registered and ready for execution:

In [ ]:
iqs.queries.list_queries()

#### Execute pre-registered query and process results

Before executing the query, we make sure that it query is served as a standing query. This means that it is evaluated once, and then results are provided instantaneously for any subsequent requests.

This might take a few seconds if you are the first user to run the following command on this IQS server instance and this model. For subsequent invocations, even by other users, this preparation will return immediately.

In [ ]:
from iqs_jupyter import schema
display(iqs.query_execution.prepare_standing_queries_on_model_compartment(
    schema.QueryFQNListWithModelCompartment(
      model_compartment = model,
      query_fq_ns = [coverage_query_main]
    )))

We will now request the results of the previusly registered query. 

In [ ]:
qResults = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
      model_compartment = model,
      query_fqn = coverage_query_main,
      query_mode = "standing"
    ))
qResults

#### Visualize query results using Pandas dataframes and Plot.ly

Setting up Cufflinks for Pandas/Plot.ly visualization:

In [ ]:
import cufflinks as cf
cf.go_offline()

The following block query renders results in Pandas dataframe format, ready for complex client-side preprocessing:

In [ ]:
qResults_df = qResults.to_data_frame().query('totalBlocks != 0')
qResults_df['pack'] = qResults_df['pack'].apply(lambda element: element.relative_element_id)
qResults_df.set_index('pack', inplace=True)
qResults_df['nonCovered'] = qResults_df['totalBlocks'] - qResults_df['stronglyCovered'] - - qResults_df['weaklyCovered'] 

Visualize the output using Plot.ly:

In [ ]:
qResults_df.sort_values(by='totalBlocks', ascending=False).drop('totalBlocks', axis=1).iplot(
    kind='bar', barmode='stack', filename='cufflinks/categorical-bar-chart/coverage',
    yTitle='Blocks in Package Traced to Requirements', title='Coverage Report',
)

#### Extract individual model elements, execute queries with parameter bindings

Descriptors of individual model elements can be extracted into Python variables from query results...

In [ ]:
first_result_element = qResults.to_list_of_matches()[0]['pack']
first_result_element

...alternatively, model element descriptors can be directly constructed using element identifiers:

In [ ]:
some_element = model.get_element_in_compartment_by_id("_18_0_2_baa02e2_1421374069634_721435_78319") 
some_element

Model elements or simple values can be used as parameter bindings to restrict the requested results:

In [ ]:
from iqs_jupyter import binding
qResults_restricted = iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(
        model_compartment = model,
        query_fqn = coverage_query_main,
        query_mode = "standing",
        parameter_binding = binding(pack=first_result_element)
    ))
qResults_restricted

### Browsing individual model elements and properties (requires connection to MMS)

This short demo section can actually be run without executing any of the code above.

First, connect to the MMS server.

In [ ]:
import iqs_jupyter # if not already imported
mms = iqs_jupyter.MMSClient(
    address = "https://mms.openmbee.org/alfresco/service",
    user = "openmbeeguest",
    password= "guest"
)

Display element information either for an element at specified commit and with specified id, or for a previously obtained model element handle. Browse through attributes and element references. Note: some element references are not shown due to MMS limitations.

In [ ]:
element_info_widget = mms.helpers.show_element_info_widget(
 org_id     = '9ff6af30-af8a-4f9d-a26b-499010ba5b6e',
 project_id = 'PROJECT-0e791c0e-16fe-422f-8f85-462ab035ce99',
 ref_id     = 'master',
 commit_id  = '0e4c90e2-ee9d-4d7e-9d90-e7fb7bb2c0c8',
 element    = '_18_0_2_baa02e2_1421374069634_721435_78319'
)
# alternatively, use the already available model element handle
# element_info_widget = mms.helpers.show_element_info_widget(some_element)
element_info_widget.display()

The element currently selected ('opened') in the widget can be obtained for further processing.

In [ ]:
element_info_widget.selected_element()

### Sandbox

## Extra section for privileged users
Do not forget to specify your privileged credentials at the top of the notebook, in the first code cell

### Repository management, indexing commits

Force the server to refresh its knowledge of commits in the repository:

In [ ]:
iqs.mms_repository.update_mms_repository()

Index another model from the repository, and then load the index into server memory:

In [ ]:
iqs.persistent_index.index_model_compartment(model)

In [ ]:
iqs.in_memory_index.load_model_compartment(model)

### Sandbox